In [ ]:
dirs = ((0,1),(1,1),(1,-1),(1,0),(-1,0),(-1,-1))

In [41]:
#= Main Function to initialize the model=#

using Agents
using Distributions
using DrWatson: @dict

include("Polimerize.jl")
# Periodic agent step function
 # TODO: bounce check für agenten in eigene funktion aussiedeln
function agent_step!(agent,model)  
if agent.id > model.Nstarts
    if agent.polym == 0   # Logic for unpolymerized agents:    
            walk!(agent, rand, model, ifempty =true)
            polimerize(agent,model) 
        end
    if agent.polym > 0 # Logic for polymerized agents:    
        if agent.GDP == false   # if not hydrolized
            agent.GDP = rand(model.P_hyd,1)[1] # hydrolize with P_hyd
            end
           depolymerize(agent,model)
        end
     end
end

agent_step! (generic function with 1 method)

In [84]:
# model-Step func:
function model_step!(model)
    model.tick += 1
    model.P_hyd = Binomial(1,model.p_hyd)                      # Prob to hydrolize from GTP to GDP: 
    model.P_polym = Binomial(1,model.p_polym)                  # Prob to Polymerize/bind to the microtuble 
    model.P_depolym_GTP = Binomial(1,1-model.p_depolym_GTP)    # Prob to depolimerize if not hydrolized (GTP - tubulin) = lower
    model.P_depolym_GDP = Binomial(1,1-model.p_depolym_GDP)    # Prob to depolimerize if hydrolized (GDP - tubulin) = higher
    
end

model_step! (generic function with 1 method)

In [43]:
@agent tubulin GridAgent{2} begin
    polym::Int64
    GDP::Bool
    # Plus::Int64
end

In [85]:
function initialize(;Nstarts, numagents, periodic,
                        griddims, p_hyd,  p_polym,
                        p_depolym_GTP,p_depolym_GDP)
   
    P_hyd = Binomial(1,p_hyd)                      # Prob to hydrolize from GTP to GDP: 
    P_polym = Binomial(1,p_polym)                  # Prob to Polymerize/bind to the microtuble 
    P_depolym_GTP = Binomial(1,1-p_depolym_GTP)    # Prob to depolimerize if not hydrolized (GTP - tubulin) = lower
    P_depolym_GDP = Binomial(1,1-p_depolym_GDP)    # Prob to depolimerize if hydrolized (GDP - tubulin) = higher
   # properties = Dict(:griddims=> griddims, :periodic=>periodic, :numagents => numagents,  # Dictionary containing 
    #                :Nstarts => Nstarts, :p_hyd => p_hyd,:p_polym => p_polym,              # model properties
     #               :p_depolym_GTP => p_depolym_GTP,:p_depolym_GDP => p_depolym_GDP,
     #               :P_hyd => P_hyd,:P_polym=> P_polym, :P_depolym_GTP=>  P_depolym_GTP,
        #            :P_depolym_GDP=> P_depolym_GDP)
    
    properties = @dict griddims numagents Nstarts p_depolym_GTP p_depolym_GDP p_polym p_hyd P_hyd P_polym  P_depolym_GTP  P_depolym_GDP
    properties[:tick] = 0
    
    space = GridSpace(griddims, periodic= periodic) # initialize Gridspace
    
    model = ABM(tubulin, space;                     # create model
        scheduler = Schedulers.randomly,
        properties = properties )
    
    id = 0# populize Space ->
    for _ in 1:Nstarts # Startingpoints:
        id += 1
        agent =  tubulin(id,griddims./2,id,false)
        if id == 1
            add_agent_pos!(agent,model)
        else
            add_agent_single!(agent,model) # add additional startingpoints to random positions 
        end
    end
    for _ in 1:numagents 
        id += 1
        agent =  tubulin(id,(1,1),0,false)
        add_agent_single!(agent,model) # add agents to random positions
    end
    return model
end

initialize (generic function with 1 method)

In [108]:
model = initialize(
    griddims =(150,150),
    periodic= true,
    Nstarts=8,
    numagents = 2580,
    p_polym=0.85,
    p_hyd= 0.01,
    p_depolym_GTP=0,
    p_depolym_GDP=0.034)

AgentBasedModel with 2588 agents of type tubulin
 space: GridSpace with size (150, 150), metric=chebyshev, periodic=true
 scheduler: randomly
 properties: Dict{Symbol, Any}(:tick => 0, :griddims => (150, 150), :P_hyd => Binomial{Float64}(n=1, p=0.01), :p_depolym_GDP => 0.034, :Nstarts => 8, :P_depolym_GTP => Binomial{Float64}(n=1, p=1.0), :P_depolym_GDP => Binomial{Float64}(n=1, p=0.966), :p_polym => 0.85, :p_hyd => 0.01, :p_depolym_GTP => 0…)

In [100]:
using InteractiveDynamics
#using CairoMakie

using GLMakie

# coloring
groupcolor(a) = a.polym >= 1 ? :red : :blue
groupmarker(a) = a.GDP == true ? :circle : :x
#GLMakie.activate!()

abm_video(
    "Microtubule_test.mp4", model, agent_step!,model_step!;
    ac = groupcolor, am = groupmarker,
    framerate = 20, frames = 2500,
    title = ("Test boundary function"))


LoadError: Screen not open!

In [109]:
using InteractiveDynamics
using GLMakie

params = Dict(
    :numagents => 0:800,
    :p_hyd => 0:0.01:1,
    :p_polym => 0:0.01:1,
    :p_depolym_GTP => 0:0.01:1,
    :p_depolym_GDP =>0:0.01:1
)
GLMakie.activate!()
# stats
# coloring
groupcolor(a) = a.polym >= 1 ? :red : :blue
groupmarker(a) = a.GDP == true ? :circle : :x
adata = [(:polym,sum), (:GDP,sum)]
alabels = ["Polimerized", "Hydrolized"]


fig, adf, mdf = abm_data_exploration(
    model, agent_step!, model_step!, params;
    ac=groupcolor, am=groupmarker, adata, alabels
)

(Scene (1600px, 800px):
  89 Plots:
    ├ Combined{Makie.poly, Tuple{Vector{Vector{Point{2, Float32}}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{String}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{Vector{Tuple{AbstractString, Point{2, Float32}}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{String}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{Vector{Tuple{AbstractString, Point{2, Float32}}}}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{String}}
    ├ Scatter{Tuple{Vec